Example case study to demonstrate the use of energiapy for solar panel reverse logistics network! Will mess around in SOLAR branch

In [2]:
import pandas 
import numpy
import sys
sys.path.append('src')

from energiapy.components.temporal_scale import TemporalScale
from energiapy.components.resource import Resource, VaryingResource
from energiapy.components.process import Process, VaryingProcess
from energiapy.components.material import Material
from energiapy.components.location import Location
from energiapy.components.transport import Transport
from energiapy.components.network import Network
from energiapy.components.scenario import Scenario
from energiapy.components.transport import Transport
from energiapy.components.result import Result 
from energiapy.utils.data_utils import get_data, make_henry_price_df, remove_outliers, load_results
from energiapy.model.formulate import formulate, Constraints, Objective
from energiapy.utils.nsrdb_utils import fetch_nsrdb_data
from energiapy.plot import plot_results, plot_scenario
from energiapy.plot.plot_results import CostY, CostX
from energiapy.model.solve import solve
from energiapy.aggregation.reduce_scenario import reduce_scenario, Clustermethod
from energiapy.aggregation.ahc import agg_hierarchial_elbow, IncludeAHC, Fit
from energiapy.aggregation.dtw import dynamic_warping, dynamic_warping_matrix, dynamic_warping_path, IncludeDTW
from energiapy.utils.data_utils import load_results
import matplotlib.pyplot as plt
from matplotlib import rc
from itertools import product

Initialize model

Temporal scale

In [3]:
scales = TemporalScale(discretization_list=[1, 365])

Resources

In [4]:
SiPV = Resource(name = 'sipv', label='Silicon Photovoltaic', price=50, basis = 'module', cons_max=130000)
glass = Resource(name = 'glass', label='Recycled glass', revenue=100, basis = 'kg', demand=True)
aluminum = Resource(name = 'al', label='Recycled aluminum', revenue=100, basis = 'kg', sell=True)
polymer = Resource(name = 'polymer', label='Recycled polymer', revenue=100, basis = 'kg', sell=True)
copper = Resource(name = 'cu', label='Recycled copper', revenue=100, basis = 'kg', sell=True)
silver = Resource(name = 'ag', label='Recycled silver', revenue=100, basis = 'kg', sell=True)
silicon = Resource(name = 'si', label='Recycled silicon', revenue=100, basis = 'kg', sell=True)

Processes

In [5]:
FRELP = Process(name = 'FRELP', label='FRELP recycling', conversion = {SiPV: -1, glass: 13.5975, polymer: 2.22, aluminum: 1.65501, silicon: 0.53835, copper: 0.17945, silver: 0.001739}, prod_max = 1300, capex = 4, vopex = 8)
ASU = Process(name = 'ASU', label='ASU recycling', conversion = {SiPV: -1, glass: 13.73625, polymer: 2.22, aluminum: 1.5651, silicon: 0.4995, copper: 0.15355, silver: 0.001369}, prod_max = 1300, capex = 17, vopex = 9)
hybrid = Process(name = 'hybrid', label='Hybrid recycling', conversion = {SiPV: -1, glass: 13.5975, polymer: 2.22, aluminum: 1.65501, silicon: 0.53835, copper: 0.15355, silver: 0.001369}, prod_max = 1300, capex = 3, vopex = 7)


FRELP_big = Process(name = 'FRELP_big', label='big FRELP recycling', conversion = {SiPV: -1, glass: 13.5975, polymer: 2.22, aluminum: 1.65501, silicon: 0.53835, copper: 0.17945, silver: 0.001739}, prod_max = 13000, capex = 3, vopex = 5)
ASU_big = Process(name = 'ASU_big', label='big ASU recycling', conversion = {SiPV: -1, glass: 13.73625, polymer: 2.22, aluminum: 1.5651, silicon: 0.4995, copper: 0.15355, silver: 0.001369}, prod_max = 13000, capex = 8, vopex = 6)
hybrid_big = Process(name = 'hybrid_big', label='big Hybrid recycling', conversion = {SiPV: -1, glass: 13.5975, polymer: 2.22, aluminum: 1.65501, silicon: 0.53835, copper: 0.15355, silver: 0.001369}, prod_max = 13000, capex = 3, vopex = 4)


Location

In [6]:
mitsubishi = Location(name='mitsubishi', processes = {FRELP, ASU, hybrid, FRELP_big, ASU_big, hybrid_big}, scales = scales, label = 'Mitsubishi recycling')

# houston = Location(name = 'HO', scales = scales, , processes = {dummy}, label = 'Houston')

Scenario

In [7]:
glassDelivery = Scenario(name = 'glassDelivery', network = mitsubishi, demand = {mitsubishi: {glass: 15000}}, scales= scales, label = 'glassDelivery', network_scale_level = 0, purchase_scale_level = 1, scheduling_scale_level = 1, demand_scale_level = 1, expenditure_scale_level = 0)

MILP formulation

In [10]:
milp_demand = formulate(scenario = glassDelivery, constraints={Constraints.INVENTORY,Constraints.COST, Constraints.PRODUCTION, Constraints.RESOURCE_BALANCE, Constraints.MODE}, objective=Objective.DEMAND)

constraint production mode
constraint production mode facility
constraint production mode binary
constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint location capex
constraint location fopex
constraint location vopex
constraint location incidental
constraint network capex
constraint network fopex
constraint network vopex
constraint network incidental
constraint nameplate inventory
constraint storage facility
constraint min storage facility
constraint nameplate production
constraint production facility
constraint min production facility
constraint inventory balance
constraint resource consumption
constraint resource purchase
constraint location production
constraint location discharge
constraint location consumption
constraint location purchase
constraint network production
constraint network discharge
constraint network consumption
constraint network purchase


In [24]:
results = solve(scenario = glassDelivery, instance=milp_demand, solver='gurobi', print_solversteps=True, name = 'results')

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter LogFile to C:\Users\NCHRIS~1\AppData\Local\Temp\tmp7iu5ytu3.log
   Prev:   Default: 
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 12117 rows, 11785 columns and 41742 nonzeros
Model fingerprint: 0x135a3144
Variable types: 7393 continuous, 4392 integer (4392 binary)
Coefficient statistics:
  Matrix range     [1e-03, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+05, 1e+05]
Found heuristic solution: objective -0.0000000
Presolve removed 12117 rows and 11785 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 2.13641e+08 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.136406593750e+0